# Load Data

In [98]:
# import modules
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import pickle
from sqlalchemy import create_engine

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import pymysql
from sshtunnel import SSHTunnelForwarder
from datetime import datetime, timedelta

import spacy

# Load pre-trained word embeddings (e.g., spaCy's medium English model)
nlp = spacy.load("en_core_web_sm")


from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import nearest_points
import geopandas as gpd
from shapely import wkt

In [99]:
ssh_host = '137.43.49.79'
ssh_port = 22
ssh_username = 'student'
ssh_password = 'Ucd-cs-2023!'

mysql_user = 'root'
mysql_password = 'Group@18'
mysql_db = 'smartcityexplorer'

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=('127.0.0.1', 3306)
) as tunnel:
    conn = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_static")
    results = cursor.fetchall()

    # Convert the results to a pandas DataFrame
    import pandas as pd
    df_venue_static = pd.DataFrame(results, columns=[column[0] for column in cursor.description])
    df_rest_static = pd.DataFrame(results, columns=[column[0] for column in cursor.description])
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_timings")
    results = cursor.fetchall()
    df_venue_timings = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_merged")
    results = cursor.fetchall()
    df_venue_merged = pd.DataFrame(results, columns=[column[0] for column in cursor.description])    
    
    # Close the database connection
    conn.close()

In [100]:
df_manhattan_zone = pd.read_csv('taxi_zones.csv')
df_manhattan_zone.head(50)


,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
0,1,0.116357,MULTIPOLYGON (((-74.18445299999996 40.69499599...,0.000782,Newark Airport,1,EWR
1,2,0.433470,MULTIPOLYGON (((-73.82337597260663 40.63898704...,0.004866,Jamaica Bay,2,Queens
2,3,0.084341,MULTIPOLYGON (((-73.84792614099985 40.87134223...,0.000314,Allerton/Pelham Gardens,3,Bronx
3,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan
4,5,0.092146,MULTIPOLYGON (((-74.17421738099989 40.56256808...,0.000498,Arden Heights,5,Staten Island
5,6,0.150491,MULTIPOLYGON (((-74.06367318899999 40.60219816...,0.000606,Arrochar/Fort Wadsworth,6,Staten Island
6,7,0.107417,MULTIPOLYGON (((-73.90413637799996 40.76752031...,0.000390,Astoria,7,Queens
7,8,0.027591,MULTIPOLYGON (((-73.92334041500001 40.77512891...,0.000027,Astoria Park,8,Queens
8,9,0.099784,MULTIPOLYGON (((-73.78502434699996 40.76103651...,0.000338,Auburndale,9,Queens
9,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan


# Manipulate Data

## Split into Hour and Day of the week

In [101]:
df_venue_merged['merged_time'] = pd.to_datetime(df_venue_merged['merged_time'])

# Add 'day_of_week' column (Monday as 0)
df_venue_merged['day_of_week'] = df_venue_merged['merged_time'].dt.dayofweek

# Add 'hour_integer' column
df_venue_merged['hour_integer'] = df_venue_merged['merged_time'].dt.hour

In [102]:
print(df_venue_merged.head(10))

   venue_id                   venue_name  busyness          venue_type  \
0  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
1  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
2  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
3  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
4  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
5  47508476  Han Dynasty Upper West Side        20  CHINESE_RESTAURANT   
6  47508476  Han Dynasty Upper West Side        25  CHINESE_RESTAURANT   
7  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
8  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
9  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   

                                    venue_address         merged_time  \
0  215 W 85th St New York, NY 10024 United States 2023-06-19 06:00:00   
1  215 W 85th St New York, NY 10024 Uni

## Grouping Venue Types

In [103]:
venue_mapping = {
    'PARK': 'Park',
    'TOURIST_DESTINATION': 'Tourist Destination',
    'MUSEUM': 'Cultural Heritage',
    'HISTORICAL':'Cultural Heritage',
    'SCENIC_POINT': 'Scenic Landmarks',
    'BRIDGE': 'Scenic Landmarks',
    'NATURE_RESERVE': 'Nature Attractions',
    'ZOO': 'Nature Attractions',
    'BOTANICAL_GARDEN': 'Nature Attractions',
    'ARTS': 'Art',
    'DESSERT':'Art',
    'CHURCH': 'Religious',
    'SYNAGOGUE':'Religious',
    'VISITOR_CENTER': 'Tourist Destination',
    'LIBRARY':'Library',
    'SHOPPING_CENTER': 'Shopping Center',
    'APPAREL':'Fashion Convenience',
    'OTHER': 'Tourist Destination',
    'SHOPPING': 'Fashion Convenience',
    'CONVENIENCE_STORE':'Neighborhood Market',
    'SUPERMARKET': 'Neighborhood Market',
    'GROCERY':'Neighborhood Market',
    'MARKET':'Neighborhood Market',
    'GIFTS': 'Gifts & Souvenirs',
    'SOUVENIR_SHOP':'Gifts & Souvenirs',
    'CHINESE_RESTAURANT': 'Chinese Restaurant',
    'AMERICAN_RESTAURANT': 'American Restaurant',
    'RESTAURANT': 'Restaurant',
    'FRENCH_RESTAURANT': 'French Restaurant',
    'BREAKFAST_RESTAURANT': 'Breakfast Restaurant',
    'ITALIAN_RESTAURANT': 'Italian Restaurant',
    'MEDITERANEAN_RESTAURANT': 'Mediteranean Restaurant',
    'SEAFOOD_RESTAURANT': 'Seafood Restaurant',
    'JAPANESE_RESTAURANT': 'Japanese Restaurant',
    'THAI_RESTAURANT': 'Thai Restaurant',
    'VEGETERIAN_RESTAURANT': 'Vegeterian Restaurant',
    'MEXICAN_RESTAURANT': 'Mexican Restaurant',
    'PIZZA_RESTAURANT': 'Pizza Restaurant',
    'ASIAN_RESTAURANT': 'Asian Restaurant',
    'INDIAN_RESTAURANT': 'Indian Restaurant',
    'BURGER_RESTAURANT': 'Burger Restaurant',
    'SUSHI_RESTAURANT': 'Sushi Restaurant',
    'RAMEN_RESTAURANT': 'Ramen Restaurant',
}

df_venue_static['venue_mod_type'] = df_venue_static['venue_type'].replace(venue_mapping)

In [104]:
venue_to_zone_dict = {}

# Iterate through each row in the DataFrame
for index, row in df_venue_static.iterrows():
    venue_type = row['venue_mod_type']
    zone = row['zone_id']
    
    # If the venue_type is already in the dictionary, append the zone to its list
    if venue_type in venue_to_zone_dict:
        venue_to_zone_dict[venue_type].append(zone)
    # If the venue_type is not in the dictionary, create a new entry with the zone as a list
    else:
        venue_to_zone_dict[venue_type] = [zone]

print(venue_to_zone_dict)

{'Chinese Restaurant': [239, 239, 239, 239, 162, 239, 142], 'American Restaurant': [142, 263, 239, 79, 161, 100, 100, 239, 238, 243, 164, 249, 107, 236, 236, 142, 142, 148, 162, 48, 48, 234, 239, 238], 'Tourist Destination': [43, 170, 186, 161, 43, 166, 144, 239, 43, 230, 164, 43, 230, 43, 246, 162, 230, 50, 43, 43, 43, 161, 236, 170, 246], 'Restaurant': [230, 142, 162, 239, 239, 142, 48, 50, 239, 164, 239, 239, 48, 142, 239, 231, 230, 239, 239, 231, 48, 239, 239, 79, 239, 142, 239, 238, 263, 239, 68, 48, 142, 236, 142, 48, 238, 48, 239, 239, 141], 'French Restaurant': [161, 161, 239, 158, 230, 79, 237, 239, 161, 230], 'Breakfast Restaurant': [239, 239], 'Italian Restaurant': [238, 163, 237, 230, 238, 239, 142, 163, 239, 142, 142, 163, 142, 142, 249, 142, 238, 239, 239, 48, 142, 141, 249, 239, 142, 230, 114], 'Cultural Heritage': [43, 24, 163, 163, 236, 239, 103, 162, 239, 43, 239, 50, 170, 161, 236, 151, 43, 239, 158, 236, 43, 230, 100, 43, 238, 75, 142, 161, 75, 236, 43], 'Fashion Co

In [105]:
df_venue_static.head(50)

,original_ven_id,hash_ven_id,latitude,longitude,zone_id,name,address,venue_type,rating,price,type_mod,zone_group,image,description,location_id,venue_mod_type
0,ven_30356e77514c7172685648526b6f77595249473851...,47508476,40.7875,-73.9764,239,Han Dynasty Upper West Side,"215 W 85th St New York, NY 10024 United States",CHINESE_RESTAURANT,4.4,2.0,Chinese Restaurant,Upper West Side,https://images.getbento.com/accounts/845ffc258...,"Trendy area with cafes, restaurants and fashio...",143368,Chinese Restaurant
1,ven_30365f61684a68514c7730526b6f775a6c6b7a7072...,88409575,40.7770,-73.9785,142,Friedman’s,"50 W 72nd St New York, NY 10023 United States",AMERICAN_RESTAURANT,4.4,2.0,American Restaurant,Upper West Side,https://lh3.googleusercontent.com/p/AF1QipM8N3...,Friedman’s is a local restaurant and bar locat...,0,American Restaurant
2,ven_303749644b67576e694b55526b6f7759685f4a4457...,1030056,40.7758,-73.9752,43,Strawberry Fields,"Q2GG+83 New York, NY 10019 United States",TOURIST_DESTINATION,4.6,-1.0,Tourist Destination,Upper West Side,https://s3.amazonaws.com/assets.centralparknyc...,Strawberry Fields is a music venue located in ...,0,Tourist Destination
3,ven_3038654c546f3936784873526b6f775a3148646771...,23854076,40.7580,-73.9853,230,Time Square,"199 W 45th St New York, NY 10036 United States",RESTAURANT,4.7,-1.0,Restaurant,Midtown Manhattan,https://upload.wikimedia.org/wikipedia/commons...,Time Square is a world-famous entertainment di...,0,Restaurant
4,ven_3039793356676f745a3570526b6f775974496c6d73...,20848781,40.7776,-73.9784,142,TAP NYC | 100% Gluten-Free Sandwiches & Açaí B...,"267 Columbus Ave New York, NY 10023 United States",RESTAURANT,4.6,2.0,Restaurant,Upper West Side,https://lh3.googleusercontent.com/p/AF1QipNByb...,TAP NYC is a 100% gluten-free sandwich and aça...,0,Restaurant
5,ven_304330524e695f70503476526b6f775a6c6e5a7133...,43331760,40.7630,-73.9767,161,The Consulate,"44 W 56th St New York, NY 10019 United States",FRENCH_RESTAURANT,4.8,-1.0,French Restaurant,Midtown Manhattan,https://s3-media0.fl.yelpcdn.com/bphoto/zvwIqh...,"The Consulate is a large, modern venue located...",0,French Restaurant
6,ven_3046564f552d3633755343526b6f77595649495368...,77654852,40.7857,-73.9726,239,The Viand,"517 Columbus Ave New York, NY 10024 United States",BREAKFAST_RESTAURANT,4.3,2.0,Breakfast Restaurant,Upper West Side,https://media-cdn.tripadvisor.com/media/photo-...,"The Viand is a restaurant located in New York,...",0,Breakfast Restaurant
7,ven_3048674a4764455f76744f526b6f77594a6f5f6176...,82820658,40.7911,-73.9740,238,Carmine's Italian Restaurant - Upper West Side,"2450 Broadway New York, NY 10024 United States",ITALIAN_RESTAURANT,4.4,2.0,Italian Restaurant,Upper West Side,https://media-cdn.tripadvisor.com/media/photo-...,Carmine's Italian Restaurant - Upper West Side...,0,Italian Restaurant
8,ven_304961777454515336474c526b6f775a4e51737357...,55261692,40.7512,-73.9749,162,The Capital Grille,"155 E 42nd St New York, NY 10017 United States",RESTAURANT,4.6,4.0,Restaurant,Midtown Manhattan,https://newyorklifestylesmagazine.com/images/c...,The Capital Grille is an upscale steakhouse wi...,0,Restaurant
9,ven_304a6c77747a366c755455526b6f77594a70705759...,57103262,40.7739,-73.9710,43,Bethesda Terrace,"New York, NY 10024 United States",HISTORICAL,4.7,-1.0,Cultural Heritage,Upper West Side,https://upload.wikimedia.org/wikipedia/commons...,Bethesda Terrace is an iconic New York City la...,10067208,Cultural Heritage


# Clearing Duplicates

In [106]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_static.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_static[df_venue_static.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [107]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_timings[df_venue_timings.duplicated(subset=['venue_id', 'day', 'opening_time', 'closing_time'], keep='first')].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 777


In [108]:
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.drop_duplicates(subset=['venue_id', 'day', 'opening_time', 'closing_time'], inplace=True))
df_venue_timings

Number of duplicate (excluding first) rows in the table is:  None


,id,venue_id,day,opening_time,closing_time
0,1,ven_30356e77514c7172685648526b6f77595249473851...,0,11,22
1,2,ven_30356e77514c7172685648526b6f77595249473851...,1,11,22
2,3,ven_30356e77514c7172685648526b6f77595249473851...,2,11,22
3,4,ven_30356e77514c7172685648526b6f77595249473851...,3,11,22
4,5,ven_30356e77514c7172685648526b6f77595249473851...,4,11,22
...,...,...,...,...,...
3250,3629,ven_77446a687a695a67306746526b6f77595a6c644878...,2,8,0
3251,3630,ven_77446a687a695a67306746526b6f77595a6c644878...,3,8,0
3252,3631,ven_77446a687a695a67306746526b6f77595a6c644878...,4,8,0
3253,3632,ven_77446a687a695a67306746526b6f77595a6c644878...,5,8,1


# Grouping Zones

In [109]:
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236,263, 262, 237, 141, 140 ],
    'Chelsea/Greenwhich market':[246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12 ],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234]
}


In [110]:
# venue_zone_grouping dictionary
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236, 263, 262, 237, 141, 140],
    'Chelsea/Greenwhich market': [246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234],
}

# Function to map zone numbers to zone groups
def map_zone_group(zone_number):
    for zone_group, zone_numbers in venue_zone_grouping.items():
        if zone_number in zone_numbers:
            return zone_group
    return 'Other'  # If zone number not found in the dictionary, assign 'Other'

# Create the 'zone_group' column based on the mapping
df_venue_static['zone_group'] = df_venue_static['zone_id'].apply(map_zone_group)

print(df_venue_static)


                                       original_ven_id  hash_ven_id  latitude  \
0    ven_30356e77514c7172685648526b6f77595249473851...     47508476   40.7875   
1    ven_30365f61684a68514c7730526b6f775a6c6b7a7072...     88409575   40.7770   
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
4    ven_3039793356676f745a3570526b6f775974496c6d73...     20848781   40.7776   
..                                                 ...          ...       ...   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
352  ven_77686b76675f51577a7053526b6f775a356b526546...     76894533   40.7694   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

In [111]:
df_venue_static.to_csv('zone_Grouping.csv', index=False)

# Extracting Only Attraction Types and Ignoring Restaurants

In [112]:
unique_type_values = df_venue_static['venue_mod_type'].unique()
unique_type_values

array(['Chinese Restaurant', 'American Restaurant', 'Tourist Destination',
       'Restaurant', 'French Restaurant', 'Breakfast Restaurant',
       'Italian Restaurant', 'Cultural Heritage', 'Fashion Convenience',
       'Mediteranean Restaurant', 'BAR', 'Park', 'Neighborhood Market',
       'Shopping Center', 'Thai Restaurant', 'Scenic Landmarks',
       'Seafood Restaurant', 'Japanese Restaurant', 'Nature Attractions',
       'Gifts & Souvenirs', 'BEER', 'Vegeterian Restaurant',
       'Mexican Restaurant', 'Pizza Restaurant', 'FISHING', 'Art',
       'Asian Restaurant', 'Indian Restaurant', 'Burger Restaurant',
       'Religious', 'Sushi Restaurant', 'Library', 'Ramen Restaurant'],
      dtype=object)

In [113]:
specific_venue_types = ['Nature Attractions', 'Shopping Center', 'Tourist Destination', 'Cultural Heritage', 'Neighborhood Market', 'Fashion Convenience', 'Library', 'Scenic Landmarks', 'Art', 'Religious', 'Park', 'Gifts & Souvenirs']

# Filter the DataFrame to only include rows with the specific venue types
df_venue_static_att = df_venue_static[df_venue_static['venue_mod_type'].isin(specific_venue_types)]

# Now 'filtered_df' contains only rows where the "Attraction_Type" is in the specified list
print(df_venue_static_att)

                                       original_ven_id  hash_ven_id  latitude  \
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
9    ven_304a6c77747a366c755455526b6f77594a70705759...     57103262   40.7739   
12   ven_30504d554d38614b6b6f78526b6f77597862677059...     91692844   40.7783   
17   ven_306c63565a476e6d385736526b6f775a39786b4a67...     54332763   40.7506   
18   ven_306d686f6679537162766c526b6f77324c69717a38...     44704230   40.8008   
..                                                 ...          ...       ...   
350  ven_7767787947374f4c683962526b6f775a3151357171...     39118662   40.7541   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

# Extracting Only Restaurants

In [114]:
restaurant_venue_types = ['Chinese Restaurant', 'American Restaurant', 'Restaurant', 'French Restaurant', 'Breakfast Restaurant', 'Italian Restaurant', 'Mediteranean Restaurant', 'Seafood Restaurant', 'Japanese Restaurant', 'Thai Restaurant', 'Vegeterian Restaurant', 'Mexican Restaurant', 'Pizza Restaurant', 'Asian Restaurant', 'Indian Restaurant', 'Burger Restaurant', 'Sushi Restaurant', 'Ramen Restaurant']

# Filter the DataFrame to only include rows with the specific venue types
df_venue_static_res = df_venue_static[df_venue_static['venue_mod_type'].isin(specific_venue_types)]

# Now 'filtered_df' contains only rows where the "Attraction_Type" is in the specified list
print(df_venue_static_res)

                                       original_ven_id  hash_ven_id  latitude  \
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
9    ven_304a6c77747a366c755455526b6f77594a70705759...     57103262   40.7739   
12   ven_30504d554d38614b6b6f78526b6f77597862677059...     91692844   40.7783   
17   ven_306c63565a476e6d385736526b6f775a39786b4a67...     54332763   40.7506   
18   ven_306d686f6679537162766c526b6f77324c69717a38...     44704230   40.8008   
..                                                 ...          ...       ...   
350  ven_7767787947374f4c683962526b6f775a3151357171...     39118662   40.7541   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

# Actual Flow

### Input

In [115]:
user_zone_input = ['Upper Manhattan','Midtown Manhattan']

In [116]:
user_input_attractions = ['Fashion Convenience']

In [117]:
user_input_res_breakfast = ['Breakfast Restaurant']
user_input_res_lunch = ['Chinese Restaurant', 'French Restaurant']
user_input_res_dinner = ['American Restaurant', 'Sushi Restaurant']

In [118]:
# Create the suggested itinerary
current_date_time = datetime.now()

# Get the current day
user_input_day = current_date_time.weekday()

In [119]:
# Step 1: Filter the data to include only restaurants
df_restaurants = df_venue_static[df_venue_static['venue_type'].str.contains('restaurant', case=False)]

In [120]:
unique_type_values_att = df_venue_static_att['venue_mod_type'].unique()
unique_type_values_att

array(['Tourist Destination', 'Cultural Heritage', 'Fashion Convenience',
       'Park', 'Neighborhood Market', 'Shopping Center',
       'Scenic Landmarks', 'Nature Attractions', 'Gifts & Souvenirs',
       'Art', 'Religious', 'Library'], dtype=object)

In [121]:
max_attractions_per_day = 3

if len(user_input_attractions) < max_attractions_per_day:
    x = max_attractions_per_day - len(user_input_attractions)
    updated_list = [num for num in unique_type_values_att if num not in user_input_attractions]
    
    similarities = []
    user_input_tag_embedding = nlp(user_input_attractions[0]).vector

    for tag in updated_list:
        tag_embedding = nlp(tag).vector
        similarity = user_input_tag_embedding.dot(tag_embedding) / (np.linalg.norm(user_input_tag_embedding) * np.linalg.norm(tag_embedding))
        similarities.append(similarity)

    sorted_indices = np.argsort(similarities)[::-1]  # Descending order
    most_similar_tags = [updated_list[i] for i in sorted_indices]
    slice_most_similar_tags = most_similar_tags[0:x]
    user_input_attractions = user_input_attractions + slice_most_similar_tags

print(user_input_attractions)

['Fashion Convenience', 'Tourist Destination', 'Cultural Heritage']


In [122]:
# Get only the attractions relevant to the user
attractions_user_filtered = df_venue_static_att[df_venue_static_att['type_mod'].isin(user_input_attractions)]

In [123]:
user_input_attractions

['Fashion Convenience', 'Tourist Destination', 'Cultural Heritage']

In [124]:
def get_time_filtered_df(data_frame, day, open_time, close_time):

    # Loop through each venue type and check for the specified condition
    for column, venue_ids in data_frame.items():
        valid_venues = []
        if column == 'original_ven_id':
            for idx, venue_id in enumerate(venue_ids):
                venue_info = df_venue_timings.loc[df_venue_timings['venue_id'] == venue_id]
                filtered_venue_info = venue_info.loc[(venue_info['day'] == day) & (venue_info['opening_time'] <= open_time) 
                                                  & (venue_info['closing_time'] >= close_time)
                                                    ]
                if not filtered_venue_info.empty:
                    valid_venues.append(venue_id)
            break

    filtered_venues = data_frame[data_frame['original_ven_id'].isin(valid_venues)]
    
    return filtered_venues

In [125]:
# Assume that each attraction has a 2 hour visiting time + a 1 hour travelling time
attraction_visiting_time_hours = 3

# Divide the day into Breakfast, Lunch, Dinner and 3 attraction slots
attraction_slot_starts = [9, 13, 16]

# Extract the attractions relevant to each timeslot
attractions_per_slot = []
for slot in attraction_slot_starts:
    attractions_per_slot.append(get_time_filtered_df(attractions_user_filtered, user_input_day, slot, slot + attraction_visiting_time_hours))

# Extract the attraction types relevant to each timeslot
attractions_type_per_slot = []
for attraction_slot in attractions_per_slot:
    attractions_type_per_slot.append(attraction_slot['venue_mod_type'].unique())

print(attractions_type_per_slot[0])
print(attractions_type_per_slot[1])
print(attractions_type_per_slot[2])

['Tourist Destination' 'Cultural Heritage' 'Fashion Convenience']
['Fashion Convenience' 'Cultural Heritage' 'Tourist Destination']
['Fashion Convenience' 'Tourist Destination' 'Cultural Heritage']


In [126]:
priority_table = pd.DataFrame({
    'Attraction': ['Parks', 'Tourist Destination', 'Cultural Heritage', 'Scenic Landmarks', 'Nature Attractions',
                   'Religious', 'Art', 'Library', 'Shopping Center', 'Fashion Convenience',
                   'Neighborhood Market', 'Gifts & Souvenirs'],
    'Opening_Time': ['9:00 AM', '9:00 AM', '11:00 AM', '9:00 AM', '10:00 AM', '11:00 AM', '9:00 AM', '9:00 AM',
                     '10:00 AM', '10:00 AM', '10:00 AM', '10:00 AM'],
    'Closing_Time': ['6:00 PM', '6:00 PM', '5:00 PM', '1:00 AM', '4:00 PM', '5:00 PM', '5:00 PM', '7:00 PM',
                     '6:00 PM', '6:00 PM', '6:00 PM', '6:00 PM']
})


# Filter priority table based on user's input
filtered_priority = priority_table[priority_table['Attraction'].isin(user_input_attractions)]


# Create the suggested itinerary
itinerary = {}
current_date_time = datetime.now()

# Get the current date
current_date = current_date_time.date()
print("WEEKDAY",current_date_time.weekday())

print("Current Date:", current_date)



current_time = pd.Timestamp(f"{current_date} 9:00 AM")
for index, row in filtered_priority.iterrows():
    attraction = row['Attraction']
    start_time = pd.Timestamp(row['Opening_Time'])
    end_time = pd.Timestamp(row['Closing_Time'])
    
#     print(end_time, current_time, start_time)

    # Check if the attraction can be visited within the day (9 AM to 7 PM)
    if end_time <= pd.Timestamp(f"{current_date} 7:00 PM") and current_time <= pd.Timestamp(f"{current_date} 7:00 PM"):
        # Handle the break from 1 am to 3 am
        if current_time.hour == 1:
            current_time = pd.Timestamp(f"{current_date} 3:00 AM")
        print(current_time, 'Current Time')
        # Calculate the visit duration for the attraction
        visit_duration = min(2, (end_time - current_time).seconds / 3600)
        print(visit_duration, 'Visit Duration')
        # Check if the visit exceeds the closing time (7 PM)
        if current_time + pd.Timedelta(hours=visit_duration) <= pd.Timestamp(f"{current_date} 7:00 PM"):
            itinerary[attraction] = f"{current_time.strftime('%I:%M %p')} - {(current_time + pd.Timedelta(hours=visit_duration)).strftime('%I:%M %p')}"
            current_time += pd.Timedelta(hours=visit_duration)
        else:
            break  # Stop adding attractions once it exceeds the closing time

# Print the suggested itinerary
print("Suggested Itinerary:")
itinerary_timing = []
opening_time = [9,11,15,17]
closing_time = [11,15,17,19]
for attraction, timing in itinerary.items():
    itinerary_timing.append(attraction)
itinerary_timing

WEEKDAY 1
Current Date: 2023-08-08
2023-08-08 09:00:00 Current Time
2 Visit Duration
2023-08-08 11:00:00 Current Time
2 Visit Duration
2023-08-08 13:00:00 Current Time
2 Visit Duration
Suggested Itinerary:


['Tourist Destination', 'Cultural Heritage', 'Fashion Convenience']

In [127]:
zone_group = []
for group in user_zone_input:
    for zone in venue_zone_grouping[group]:
        zone_group.append(zone)
zone_group

[128,
 127,
 243,
 120,
 244,
 116,
 42,
 152,
 41,
 74,
 75,
 50,
 48,
 230,
 163,
 161,
 162,
 229,
 233,
 164,
 170,
 137,
 224,
 107,
 234]

In [128]:
# Filter the restaurant DataFrame based on the zone IDs
df_filtered_restaurants = df_restaurants[df_restaurants['zone_id'].isin(zone_group)]

# Print the filtered DataFrame
print(df_filtered_restaurants)

                                       original_ven_id  hash_ven_id  latitude  \
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
5    ven_304330524e695f70503476526b6f775a6c6e5a7133...     43331760   40.7630   
8    ven_304961777454515336474c526b6f775a4e51737357...     55261692   40.7512   
15   ven_3068674e6f56796343637a526b6f77597431776642...     45765601   40.7639   
23   ven_342d4f33343573744c7a2d526b6f775a4277454754...      2779193   40.7545   
24   ven_34416b6a74734635334274526b6f775964314e4669...     61577324   40.7644   
45   ven_3855585f5861474e706f34526b6f77593930793336...     63592504   40.7646   
48   ven_385a743953644e70587637526b6f7759395f534f53...     75402097   40.7598   
66   ven_4148494f684649484f4477526b6f775a4e68634d42...     10162275   40.7457   
77   ven_4168437230416a45783158526b6f7759526c2d6f53...     19253272   40.7575   
80   ven_417239716752736e6c447a526b6f7759745078672d...     52953994   40.7611   
89   ven_454b716e73544e6e535

In [129]:
num_top_restaurants = 5  # just example 

# # Sort the DataFrame by rating in descending order and then by price in ascending order
# df_filtered_restaurants = df_filtered_restaurants.sort_values(by=['rating', 'price'], ascending=[False, True])

# # Display the top restaurants
# print(df_filtered_restaurants.head(num_top_restaurants))

In [130]:
print(df_filtered_restaurants)

                                       original_ven_id  hash_ven_id  latitude  \
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
5    ven_304330524e695f70503476526b6f775a6c6e5a7133...     43331760   40.7630   
8    ven_304961777454515336474c526b6f775a4e51737357...     55261692   40.7512   
15   ven_3068674e6f56796343637a526b6f77597431776642...     45765601   40.7639   
23   ven_342d4f33343573744c7a2d526b6f775a4277454754...      2779193   40.7545   
24   ven_34416b6a74734635334274526b6f775964314e4669...     61577324   40.7644   
45   ven_3855585f5861474e706f34526b6f77593930793336...     63592504   40.7646   
48   ven_385a743953644e70587637526b6f7759395f534f53...     75402097   40.7598   
66   ven_4148494f684649484f4477526b6f775a4e68634d42...     10162275   40.7457   
77   ven_4168437230416a45783158526b6f7759526c2d6f53...     19253272   40.7575   
80   ven_417239716752736e6c447a526b6f7759745078672d...     52953994   40.7611   
89   ven_454b716e73544e6e535

In [131]:
print(df_venue_merged)

        venue_id                   venue_name  busyness          venue_type  \
0       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
1       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
2       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
3       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
4       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
...          ...                          ...       ...                 ...   
231949  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231950  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231951  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231952  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231953  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   

                                            venue_a

In [132]:
# Extract restaurant busyness
df_venue_merged['venue_id'] = df_venue_merged['venue_id'].astype(int)
column_length = df_filtered_restaurants.shape[0]
busyness_column = np.zeros(column_length)

test = True

for column, venue_ids in df_filtered_restaurants.items():
    if column == 'hash_ven_id':
        for idx, venue_hash_id in enumerate(venue_ids):
            specific_venue_df = df_venue_merged.loc[df_venue_merged['venue_id'] == venue_hash_id]
            busyness_column[idx] = specific_venue_df['busyness'].mean() 
        break

df_filtered_restaurants['busyness'] = busyness_column
print(df_filtered_restaurants['busyness'])

3       9.643939
5      16.212121
8      20.212121
15     14.098485
23     11.681818
24     52.462121
45     19.848485
48     20.628788
66     18.037879
77     29.371212
80     15.500000
89     20.303030
115    19.750000
142     7.401515
156     8.121212
159    12.234848
167    21.712121
170    27.272727
181    21.204545
184    24.409091
232    19.257576
270     8.113636
273    13.772727
285    17.553030
302    19.757576
307    23.666667
309    10.045455
319    17.431818
321    34.166667
322    22.492424
330    28.977273
334    19.333333
339    27.696970
Name: busyness, dtype: float64


In [133]:
introvert_mode = False  # We have it off by default?

# We normalise the price and rating columns to the range 0-1
df_filtered_restaurants['normalized_price'] = 1 - ((df_filtered_restaurants['price'] - df_filtered_restaurants['price'].min()) / (df_filtered_restaurants['price'].max() - df_filtered_restaurants['price'].min()))
df_filtered_restaurants['normalized_rating'] = (df_filtered_restaurants['rating'] - df_filtered_restaurants['rating'].min()) / (df_filtered_restaurants['rating'].max() - df_filtered_restaurants['rating'].min())

# We then define the weights for the rating and price. (These should add up to 1.)
rating_weight = 0.33
price_weight = 0.33
busyness_weight = 0.33

# We calculate the composite score here ? (if introvert mode is on, we take that into account)
df_filtered_restaurants['normalized_busyness'] = 1 - ((df_filtered_restaurants['busyness'] - df_filtered_restaurants['busyness'].min()) / (df_filtered_restaurants['busyness'].max() - df_filtered_restaurants['busyness'].min()))

if introvert_mode:
    df_filtered_restaurants['composite_score'] = df_filtered_restaurants['normalized_rating'] * rating_weight + df_filtered_restaurants['normalized_price'] * price_weight + df_filtered_restaurants['normalized_busyness'] * busyness_weight
else:
    df_filtered_restaurants['composite_score'] = df_filtered_restaurants['normalized_rating'] * rating_weight + df_filtered_restaurants['normalized_price'] * price_weight

# Compensate for user restaurant selections
df_filtered_restaurants['breakfast_score'] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_breakfast)) * 0.5 + df_filtered_restaurants['composite_score'] * price_weight
df_filtered_restaurants['lunch_score'    ] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_lunch    )) * 0.5 + df_filtered_restaurants['lunch_score']     * price_weight
df_filtered_restaurants['dinner_score'   ] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_dinner   )) * 0.5 + df_filtered_restaurants['dinner_score']    * price_weight

# Then we can sort the DataFrame by the composite score in descending order
df_filtered_rest_breakfast = get_time_filtered_df(df_filtered_restaurants, user_input_day, 8, 9).sort_values('breakfast_score', ascending=False)
df_filtered_rest_lunch     = get_time_filtered_df(df_filtered_restaurants, user_input_day, 12, 13).sort_values('lunch_score', ascending=False)
df_filtered_rest_dinner    = get_time_filtered_df(df_filtered_restaurants, user_input_day, 19, 20).sort_values('dinner_score', ascending=False)

# Display the top restaurants
print('BREAKFAST: (Preferred ',user_input_res_breakfast,') ',df_filtered_rest_breakfast.head(num_top_restaurants))
print('LUNCH: (Preferred ',user_input_res_lunch,') ',df_filtered_rest_lunch.head(num_top_restaurants))
print('DINNER: (Preferred ',user_input_res_dinner,') ',df_filtered_rest_dinner.head(num_top_restaurants))

KeyError: 'lunch_score'

In [ ]:
user_venue_per_type_dict = {}
for venue_type in itinerary_timing:
    matched_zones = df_venue_static_att[df_venue_static_att['venue_mod_type'] == venue_type]['zone_id']
    matching_zones = matched_zones[matched_zones.isin(zone_group)]
    result_df = df_venue_static_att[df_venue_static_att['zone_id'].isin(matching_zones)]['original_ven_id']
    user_venue_per_type_dict[venue_type] = list(result_df)

# Print the resulting dictionary
print(user_venue_per_type_dict)

In [ ]:
user_restaurant_per_zone_dict = {}
for venue_type in user_input_venues:
    matched_zones = df_restaurants[df_restaurants['venue_mod_type'] == venue_type]['zone_id']
    matching_zones = matched_zones[matched_zones.isin(zone_group)]
    result_df = df_restaurants[df_restaurants['zone_id'].isin(matching_zones)]['original_ven_id']
    user_restaurant_per_zone_dict[venue_type] = list(result_df)

In [ ]:
# Need to check for empty Type

In [ ]:
zone_type_dict = {}
for venue_type in itinerary_timing:
    matched_zones = df_venue_static_att[df_venue_static_att['venue_mod_type'] == venue_type]['zone_id'].unique()
    zone_type_dict[venue_type] = list(matched_zones)

# Print the resulting dictionary
print(zone_type_dict)

In [ ]:
types_with_zero_zones = []

# Iterate through the venue_type_dict
for venue_type, zones in zone_type_dict.items():
    if len(zones) == 0:
        types_with_zero_zones.append(venue_type)

print("Venue types with 0 zones:", types_with_zero_zones)

In [ ]:
def find_distance_between_zones(zone1_polygon, zone2_polygon):
    # Find the nearest points between the two polygons
    nearest_points_result = nearest_points(wkt.loads(zone1_polygon), wkt.loads(zone2_polygon))

    # Calculate the distance between the nearest points
    distance = nearest_points_result[0].distance(nearest_points_result[1])
    return distance

In [ ]:
filled_type_with_zero_zone = {}

if len(types_with_zero_zones) > 0:
    for ven_type in types_with_zero_zones:
        venue_to_zone_dict_copy = list(set(venue_to_zone_dict[i]))
        print(lenvenue_to_zone_dict_copy)
        if len(venue_to_zone_dict_copy) < 0:
            #get all venue id of each zone and push it to user_venue_per_type_dict of that type
            print('okay')
        else:
            zone_between_dist = []
            for user_zone in zone_group:
                for venue_zone in venue_to_zone_dict_copy:
                    zone1_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == user_zone]['the_geom'].iloc[0]
                    zone2_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == venue_zone]['the_geom'].iloc[0]
                    distance = find_distance_between_zones(zone1_polygon, zone2_polygon)
                    zone_between_dist.append((user_zone, venue_zone, distance)) 
            sorted_zone_between_dist = sorted(zone_between_dist, key=lambda x: x[2])[:3]
            new_zone = [df_venue_static_att[
        (df_venue_static_att['zone_id'] == item[1]) &
        (df_venue_static_att['venue_mod_type'] == ven_type)
    ]['original_ven_id'].tolist() for item in sorted_zone_between_dist]
    
    print(new_zone)
    
    filled_type_with_zero_zone[ven_type] = list(set(item for sublist in new_zone for item in sublist))
    filled_type_with_zero_zone

In [ ]:
for key in user_venue_per_type_dict.keys():
    # Check if the value of the current key is an empty array
    if len(user_venue_per_type_dict[key]) == 0:
        # Check if the key exists in dictionary B
        if key in filled_type_with_zero_zone:
            # Replace the value in A with the value from B
            user_venue_per_type_dict[key] = filled_type_with_zero_zone[key]
print(user_venue_per_type_dict)

In [ ]:
time_index = 0
filtered_venues = {}

# Loop through each venue type and check for the specified condition
for venue_type, venue_ids in user_venue_per_type_dict.items():
    valid_venues = []
    for venue_id in venue_ids:
        venue_info = df_venue_timings.loc[df_venue_timings['venue_id'] == venue_id]
        filtered_venue_info = venue_info.loc[(venue_info['day'] == user_input_day) & (venue_info['opening_time'] <= opening_time[time_index]) 
                                              & (venue_info['closing_time'] >= closing_time[time_index])
                                            ]
        if not filtered_venue_info.empty:
            valid_venues.append(venue_id)
    if valid_venues:
        filtered_venues[venue_type] = valid_venues
        time_index+=1
print(filtered_venues)


In [ ]:
manipulated_venues = {}

if introvert_mode:
    weight_rating = 0.3
    weight_busyness = 0.7
else:
    weight_rating = 0.6
    weight_busyness = 0.4

# With introvert mode
for venue_type, venue_ids in user_venue_per_type_dict.items():
    valid_venues = []
    for venue_id in venue_ids:
        venue_hash_id = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['hash_ven_id']
        venue_rating = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['rating'].item()
        venue_hash_id = int(venue_hash_id)
        df_venue_merged['venue_id'] = df_venue_merged['venue_id'].astype(int)
        specific_venue_df = df_venue_merged.loc[df_venue_merged['venue_id'] == venue_hash_id]
        average_busyness = specific_venue_df['busyness'].mean() 
        
        composite_score = (weight_rating * venue_rating) + (weight_busyness * average_busyness)
        
        valid_venues.append((venue_id, venue_rating, average_busyness, composite_score))
    if valid_venues:
        manipulated_venues[venue_type] = valid_venues

manipulated_venues

In [ ]:
def print_top_3_recommended_restaurants(data_frame):
    for rank, ind in enumerate(data_frame.index):
        venue_id = data_frame['original_ven_id'][ind]
        rating = data_frame['rating'][ind]
        busyness = data_frame['busyness'][ind]
        score = data_frame['composite_score'][ind]
        print(f"Rank {rank + 1}: Venue ID: {venue_id}, Rating: {rating}, Busyness: {busyness}, Score: {score}")
        if rank > 1:
            break;

In [ ]:
top_3_venues = {}

# Loop through each venue type and its venues
for venue_type, venue_data in manipulated_venues.items():
    # Sort the venues based on the composite score (fourth element in the tuple, index 3)
    if len(venue_data) > 3:
        sorted_venues = sorted(venue_data, key=lambda x: x[3], reverse=True)
    
        # Keep only the top 3 venues for each venue type
        top_3_venues[venue_type] = sorted_venues[:3]
    else:
        top_3_venues[venue_type] = venue_data

# Display the top 3 venues for each venue type
for venue_type, top_venues in top_3_venues.items():
    print(f"Venue Type: {venue_type}")
    for rank, (venue_id, rating, busyness, score) in enumerate(top_venues, start=1):
        print(f"Rank {rank}: Venue ID: {venue_id}, Rating: {rating}, Busyness: {busyness}, Score: {score}")
    print()

# Display the top 3 restaurants for each type
print(f"Restaurant Type: Breakfast")
print_top_3_recommended_restaurants(df_filtered_rest_breakfast)
print()
print(f"Restaurant Type: Lunch")
print_top_3_recommended_restaurants(df_filtered_rest_lunch)
print()
print(f"Restaurant Type: Dinner")
print_top_3_recommended_restaurants(df_filtered_rest_dinner)

print()